## Brief intro

This notebook is a draft of the fly scripts of converting spatial locations into decimal degree and geojson format.

Discover as you go through the code.

In [ ]:
%load_ext lab_black
import pandas as pd
import json

### 1. geojson Polygon creation from string columns

In [ ]:
# A cell of some custom functions
def degree_to_decimal(input: str) -> float:
    degree_vals = input.split()
    assert len(degree_vals) == 3, "Expected length is 3"
    num_vals = []
    for val in degree_vals:
        num_vals.append(get_only_numerical(val))
    return num_vals[0] + num_vals[1] / 60 + num_vals[2] / 3600


def get_only_numerical(input: str) -> float:
    import re

    return float(re.findall(r"[-+]?\d*\.\d+|\d+", input)[0])

In [ ]:
# Reading the MS-Excel file
fc_df = pd.read_excel("source_file.xlsx", sheet_name="bag_coordinate")
fc_df.head()

In [ ]:
discover = fc_df["Urgurug"][0].split()
print(f"first: {discover[0]} \nsecond: {discover[1]} \nthird:{discover[2]} ")

In [ ]:
fc_df["Lat"] = fc_df["Urgurug"].apply(lambda x: degree_to_decimal(x))
fc_df["Long"] = fc_df["Urtrag"].apply(lambda x: degree_to_decimal(x))

In [ ]:
df = fc_df[["Bag", "Lat", "Long"]].copy()
df.info()

In [ ]:
df_grpd = df.groupby(by="Bag")

In [ ]:
features = []

for name, group in df_grpd:
    print(name)
    raw_coordinates = []
    for row_index, row in group.iterrows():
        raw_coordinates.append([row["Long"], row["Lat"]])

    features.append(
        {
            "type": "Feature",
            "geometry": {"type": "Polygon", "coordinates": [raw_coordinates]},
            "properties": {"bagiin_ner": name},
        }
    )

In [ ]:
geojson = {"type": "FeatureCollection", "features": features}

In [ ]:
with open("polygon.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False)

### 2. Point creation from string and seperate columns

In [ ]:
clean_df = pd.read_excel("hudag.xlsx", index_col=False)
clean_df.info()

In [ ]:
features = []
for row_ind, row in clean_df.iterrows():
    bag_ner = " ".join(row["bagiin_ner"].split())
    nname = row["bairshil_hudgiin_ner"]
    ezemshigch = row["ezemshig_ashiglagch"]

    features.append(
        {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [row["Long"], row["Lat"]]},
            "properties": {
                "bagiin_ner": bag_ner,
                "name": nname,
                "ezemshigch": ezemshigch
            },
        }
    )

In [ ]:
hudag_geojson = {"type": "FeatureCollection", "features": features}

In [ ]:
with open("hudag.geojson", "w", encoding="utf-8") as f:
    json.dump(hudag_geojson, f, ensure_ascii=False)

### 3. Point and polygon creation from string and seperate columns

In [ ]:
fc_df = pd.read_excel("source_file.xlsx", sheet_name="rashaan")
fc_df.info()

In [ ]:
fc_df["Lat"] = fc_df["Xd"] + (fc_df["Xm"] / 60) + (fc_df["Xs"] / 3600)
fc_df["Long"] = fc_df["Yd"] + (fc_df["Ym"] / 60) + (fc_df["Ys"] / 3600)

In [ ]:
fc_df.columns = [
    "dd",
    "rashaanii_ner",
    "bag",
    "ezemshil",
    "Xd",
    "Xm",
    "Xs",
    "Yd",
    "Ym",
    "Ys",
    "H",
    "zoriulalt",
    "hamgaalalt",
    "hamgaalaltiin_turul",
    "hereglee",
    "tailbar",
    "Lat",
    "Long",
]

In [ ]:
fc_grpd = fc_df.groupby("rashaanii_ner")

In [ ]:
points = []
polygons = []

point_features = []
polygon_features = []

for name, grp in fc_grpd:
    if len(grp) == 1:
        points.append(grp)

        for row_ind, row in grp.iterrows():
            bag_ner = row["bag"]
            nname = row["rashaanii_ner"]
            ezemshigch = row["ezemshil"]

            point_features.append(
                {
                    "type": "Feature",
                    "geometry": {
                        "type": "Point",
                        "coordinates": [row["Long"], row["Lat"]],
                    },
                    "properties": {
                        "bagiin_ner": bag_ner,
                        "name": nname,
                        "ezemshigch": ezemshigch,
                    },
                }
            )
    else:
        polygons.append(grp)
        raw_coordinates = []
        for row_index, row in grp.iterrows():

            bag_ner = row["bag"]
            nname = row["rashaanii_ner"]
            ezemshigch = row["ezemshil"]
            raw_coordinates.append([row["Long"], row["Lat"]])

        polygon_features.append(
            {
                "type": "Feature",
                "geometry": {"type": "Polygon", "coordinates": [raw_coordinates]},
                "properties": {
                    "bagiin_ner": bag_ner,
                    "name": nname,
                    "ezemshigch": ezemshigch,
                },
            }
        )

In [ ]:
pd.concat(points).reset_index(drop=True).to_csv(
    "rashaan_points.csv", encoding="utf-8", index=False
)

In [ ]:
point_geojson = {"type": "FeatureCollection", "features": point_features}

polygon_geojson = {"type": "FeatureCollection", "features": polygon_features}

In [ ]:
with open("rashaan_point.geojson", "w", encoding="utf-8") as f:
    json.dump(point_geojson, f, ensure_ascii=False)

with open("rashaan_polygon.geojson", "w", encoding="utf-8") as f:
    json.dump(polygon_geojson, f, ensure_ascii=False)

### 4. Point and polygon creation from string and seperate columns(different sheet)

In [ ]:
fc_df = pd.read_excel("source_file.xlsx", sheet_name="bulag")
fc_df.info()

In [ ]:
fc_df["Lat"] = fc_df["Xd"] + (fc_df["Xm"] / 60) + (fc_df["Xs"] / 3600)
fc_df["Long"] = fc_df["Yd"] + (fc_df["Ym"] / 60) + (fc_df["Ys"] / 3600)

In [ ]:
fc_df.columns = [
    "dd",
    "bulgiin_ner",
    "bagiin_ner",
    "ezemshil",
    "Xd",
    "Xm",
    "Xs",
    "Yd",
    "Ym",
    "Ys",
    "H",
    "zoriulalt",
    "hamgaalalt",
    "hashlagiin_turul_hiits",
    "bulgiin_gorim",
    "bulgiin_turul",
    "bulgiin_undarga",
    "tailbar",
    "Lat",
    "Long",
]

In [ ]:
fc_grpd = fc_df.groupby("bulgiin_ner")

In [ ]:
points = []
polygons = []

point_features = []
polygon_features = []

for name, grp in fc_grpd:
    if len(grp) == 1:
        points.append(grp)

        for row_ind, row in grp.iterrows():
            bag_ner = row["bagiin_ner"]
            nname = row["bulgiin_ner"]
            ezemshigch = row["ezemshil"]
            zoriulalt = row["zoriulalt"]
            hamgaalalt = row["hamgaalalt"]
            hashlaga = row["hashlagiin_turul_hiits"]
            bulag_gorim = row["bulgiin_gorim"]
            bulag_turul = row["bulgiin_turul"]
            bulag_undarga = row["bulgiin_undarga"]
            tailbar = row["tailbar"]

            point_features.append(
                {
                    "type": "Feature",
                    "geometry": {
                        "type": "Point",
                        "coordinates": [row["Long"], row["Lat"]],
                    },
                    "properties": {
                        "bagiin_ner": bag_ner,
                        "name": nname,
                        "ezemshigch": ezemshigch,
                        "zoriulal": zoriulalt,
                        "hamgaalalt": hamgaalalt,
                        "hashlaga": hashlaga,
                        "bulgiin_gorim": bulag_gorim,
                        "bulgiin_turul": bulag_turul,
                        "bulgiin_undarga": bulag_undarga,
                        "tailbar": tailbar,
                    },
                }
            )
    else:
        polygons.append(grp)
        raw_coordinates = []
        for row_index, row in grp.iterrows():

            bag_ner = row["bagiin_ner"]
            nname = row["bulgiin_ner"]
            ezemshigch = row["ezemshil"]

            zoriulalt = row["zoriulalt"]
            hamgaalalt = row["hamgaalalt"]
            hashlaga = row["hashlagiin_turul_hiits"]
            bulag_gorim = row["bulgiin_gorim"]
            bulag_turul = row["bulgiin_turul"]
            bulag_undarga = row["bulgiin_undarga"]
            tailbar = row["tailbar"]

            raw_coordinates.append([row["Long"], row["Lat"]])

        polygon_features.append(
            {
                "type": "Feature",
                "geometry": {"type": "Polygon", "coordinates": [raw_coordinates]},
                "properties": {
                    "bagiin_ner": bag_ner,
                    "name": nname,
                    "ezemshigch": ezemshigch,
                    "zoriulal": zoriulalt,
                    "hamgaalalt": hamgaalalt,
                    "hashlaga": hashlaga,
                    "bulgiin_gorim": bulag_gorim,
                    "bulgiin_turul": bulag_turul,
                    "bulgiin_undarga": bulag_undarga,
                    "tailbar": tailbar,
                },
            }
        )

In [ ]:
pd.concat(points).reset_index(drop=True).to_csv(
    "bulag_points.csv", encoding="utf-8", index=False
)

In [ ]:
bulag_point_geojson = {"type": "FeatureCollection", "features": point_features}

bulag_polygon_geojson = {"type": "FeatureCollection", "features": polygon_features}

In [ ]:
with open("bulag_point.geojson", "w", encoding="utf-8") as f:
    json.dump(bulag_point_geojson, f, ensure_ascii=False)

with open("bulag_polygon.geojson", "w", encoding="utf-8") as f:
    json.dump(bulag_polygon_geojson, f, ensure_ascii=False)

### 5. Polygon creation from string and seperate columns(different sheet)

In [ ]:
fc_df = pd.read_excel("source_file.xlsx", sheet_name="gol")
fc_df.info()

In [ ]:
fc_df["Lat"] = fc_df["Xd"] + (fc_df["Xm"] / 60) + (fc_df["Xs"] / 3600)
fc_df["Long"] = fc_df["Yd"] + (fc_df["Ym"] / 60) + (fc_df["Ys"] / 3600)

In [ ]:
fc_df.columns = [
    "dd",
    "goliin_ner",
    "bairshil",
    "Xd",
    "Xm",
    "Xs",
    "Yd",
    "Ym",
    "Ys",
    "H",
    "ashiglalt_heregleenii_medeelel",
    "ariun_tsever_hamgaalalt",
    "goliin_tejeellin_gorim",
    "goliin_tsudgadag_gol_murun_nuuriin_ner",
    "goliin_gorim",
    "tailbar",
    "Lat",
    "Long",
]

In [ ]:
fc_grpd = fc_df.groupby("goliin_ner")

In [ ]:
points = []
polygons = []

point_features = []
polygon_features = []

for name, grp in fc_grpd:
    if len(grp) == 1:
        points.append(grp)

        for row_ind, row in grp.iterrows():
            nname = row["goliin_ner"]
            bairshil = row["bairshil"]
            hereglee = row["ashiglalt_heregleenii_medeelel"]
            hamgaalalt = row["ariun_tsever_hamgaalalt"]
            # hashlaga = row["hashlagiin_turul_hiits"]
            tejeeliin_gorim = row["goliin_tejeellin_gorim"]
            gol_tsutgal = row["goliin_tsudgadag_gol_murun_nuuriin_ner"]
            goliin_gorim = row["goliin_gorim"]
            tailbar = row["tailbar"]

            point_features.append(
                {
                    "type": "Feature",
                    "geometry": {
                        "type": "Point",
                        "coordinates": [row["Long"], row["Lat"]],
                    },
                    "properties": {
                        "name": nname,
                        "bairshil": bairshil,
                        "hereglee": hereglee,
                        "hamgaalalt": hamgaalalt,
                        # "hashlaga": hashlaga,
                        "tejeeliin_gorim": tejeeliin_gorim,
                        "gol_tsutgal": gol_tsutgal,
                        "goliin_gorim": goliin_gorim,
                        "tailbar": tailbar,
                    },
                }
            )
    else:
        polygons.append(grp)
        raw_coordinates = []
        for row_index, row in grp.iterrows():

            nname = row["goliin_ner"]
            bairshil = row["bairshil"]
            hereglee = row["ashiglalt_heregleenii_medeelel"]
            hamgaalalt = row["ariun_tsever_hamgaalalt"]
            # hashlaga = row["hashlagiin_turul_hiits"]
            tejeeliin_gorim = row["goliin_tejeellin_gorim"]
            gol_tsutgal = row["goliin_tsudgadag_gol_murun_nuuriin_ner"]
            goliin_gorim = row["goliin_gorim"]
            tailbar = row["tailbar"]

            raw_coordinates.append([row["Long"], row["Lat"]])

        polygon_features.append(
            {
                "type": "Feature",
                "geometry": {"type": "Polygon", "coordinates": [raw_coordinates]},
                "properties": {
                    "name": nname,
                    "bairshil": bairshil,
                    "hereglee": hereglee,
                    "hamgaalalt": hamgaalalt,
                    # "hashlaga": hashlaga,
                    "tejeeliin_gorim": tejeeliin_gorim,
                    "gol_tsutgal": gol_tsutgal,
                    "goliin_gorim": goliin_gorim,
                    "tailbar": tailbar,
                },
            }
        )

In [ ]:
gol_point_geojson = {"type": "FeatureCollection", "features": point_features}

gol_polygon_geojson = {"type": "FeatureCollection", "features": polygon_features}

In [ ]:
with open("gol_point.geojson", "w", encoding="utf-8") as f:
    json.dump(gol_point_geojson, f, ensure_ascii=False)

with open("gol_polygon.geojson", "w", encoding="utf-8") as f:
    json.dump(gol_polygon_geojson, f, ensure_ascii=False)